In [ ]:
source = open("05test.txt", "r").read()
sections = source.split("\n\n")

seed_list = sections[0];
seeds = seed_list.split(" ")[1:]
maps = sections[1:]

location_maps = []
for map in maps:
    lines = map.split("\n")
    label = lines[0]
    mappings = lines[1:]
    location_map = {"label": label, "mappings": []}

    for line in mappings:
        target_start, seed_start, seed_range = line.split(" ")
        seed_start = int(seed_start)
        target_start = int(target_start)
        seed_range = int(seed_range)
        location_map["mappings"].append((seed_start, target_start, seed_range))
    location_map["mappings"] = sorted(location_map["mappings"], key=lambda x: x[0])
    location_maps.append(location_map)
def apply_map(start, location_map):
    for mapping in location_map["mappings"]:
        seed_start, target_start, seed_range = mapping
        if start >= seed_start and start < seed_start + seed_range:
            return target_start + (start - seed_start)
    return start
ends = []
for seed in seeds:
    start = int(seed)
    for location_map in location_maps:
        start = apply_map(start, location_map)
    ends.append(start)
ends
min(ends)

In [ ]:
location_maps

In [ ]:
import numpy as np 
seed_ranges = (np.array_split([int(x) for x in seeds], 2))
print("starting with seed ranges", seed_ranges)
iterations = 0
for location_map in location_maps:
    windows = []
    print ("-----" + location_map["label"] + "-----")
    print("----- with seed ranges", seed_ranges)
    for seed_start, seed_range in seed_ranges:
        #evaluate if seed range is in location range
        seed_max = seed_start + seed_range
        print("STARTING with seed range of", seed_start, seed_max)
        print(location_map["mappings"])
        found = False
        do_loop = True
        while do_loop == True: 
            seed_max = seed_start + seed_range
           
            for location_seed_start, location_start, location_range in location_map["mappings"]:
                 iterations = iterations + 1
            if iterations > 100: 
                exit()
                break
                print("raw location info", location_seed_start, location_start, location_range)
                location_seed_max = location_seed_start + location_range
                location_offset = location_start - location_seed_start
                print("comparing seed range of", seed_start, seed_max, "with location range of", location_seed_start, location_seed_max, location_offset)
                #window possibilities
                if( seed_start < location_seed_start):
                    if(seed_max < location_seed_start):
                        print("A SEED starts and ends before location", seed_start, seed_max, "with location range of", location_seed_start, location_seed_max)
                        do_loop = False
                        found = True
                        break;
                    else:
                        windows.append((seed_start, location_seed_start - 1))
                        seed_start = location_seed_start
                        seed_range = seed_max - location_seed_start
                        found = True
                if(seed_start >= location_seed_start and seed_start < location_seed_max):
                    # Starts within
                    if(seed_max <= location_seed_max):
                        # within
                        windows.append((seed_start + location_offset, seed_range))
                        do_loop = False
                        found = True
                        break;
                    else:
                         windows.append((seed_start + location_offset, location_seed_max - seed_start))
                         seed_start = location_seed_max
                         seed_range = seed_max - location_seed_max
                         seed_max = seed_start + seed_range
                        # ends after
                        # found = False
                if(seed_start >= location_seed_max): 
                     found = False
                    # print("Total miss", seed_start, seed_max, "with location range of", location_seed_start, location_seed_max)
                if found == True : break
        if found == False:
            windows.append((seed_start, seed_range)) # If I miss all windows entirely that means theres no change to be had
            print("NOBODY matched, so added window of", [windows[-1]])
    seed_ranges = list(set(windows))
    print("END seed ranges at the end of " + location_map["label"], seed_ranges )
seed_ranges
print("seed ranges at the end", seed_ranges)
print("Start/stops at the end", [(x[0], x[0] + x[1] -1 ) for x in seed_ranges if x[1] > 0])
new_seeds = np.array([(x[0], x[0] + x[1] -1 ) for x in seed_ranges if x[1] > 0]).flatten()
print("new seeds", new_seeds)
min(new_seeds)

# new_ends = []
# # print(new_seeds)
# for seed in new_seeds:
#     start = int(seed)
# #     for location_map in location_maps:
# #         # print("Starting on map: " + location_map["label"] + " with start: " + str(start) + "and mappings" + str(location_map["mappings"]))
#     start = apply_map(start, location_maps[-1])
#     print("Ended with ", start)
#     new_ends.append(start)
# # new_ends
# min(new_ends)